In [1]:
# 서울버스정류장
import requests, bs4
import mysql.connector
from lxml import html
from urllib.request import Request,urlopen
from urllib.parse import urlencode,quote_plus
from bs4 import BeautifulSoup
from xml.etree.ElementTree import parse
from datetime import datetime
from ServiceKey import public_serviceKey, mySQL_Connect

mydb = mySQL_Connect()

cursor = mydb.cursor(prepared=True)

# 2021.09.17일 수집 완료
query = ("select seoulsttsName from public_seoulbusstts ps where 3086 <= seq and seq <= 4100")

cursor.execute(query)

location_list = []

for i in cursor:
    location_list.append(i)
    
now = datetime.now()
nowDate = now.strftime('%Y%m%d')

cnt = 1
for i in location_list:
    seoulsttsName = i
    
    characters = "(',')"

    join_data = ''.join( x for x in seoulsttsName if x not in characters)
    print("조회날짜 : {} | 순번 : {} | start {} public_seoulbusstts".format(nowDate, cnt, join_data))
    url = 'http://ws.bus.go.kr/api/rest/stationinfo/getStationByName'
    queryParams = '?' + urlencode({
        quote_plus('serviceKey') : public_serviceKey(),
        quote_plus('stSrch') : join_data
    })

    response = requests.get(url + queryParams).text.encode('utf-8')
    xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

    rows = xmlobj.findAll('itemList')
    resultCode = xmlobj.findAll('headerCd')

    if resultCode[0].text == '7':
        print("{} 번째부터 조회수 제한이 걸렸습니다.".format(seoulsttsName))
        mydb.close()
        break
    cnt += 1
    for j in rows:
        item = j
        arsId = item.findAll('arsId')
        stId = item.findAll('stId')
        stNm = item.findAll('stNm')
        lat = item.findAll('tmY')
        lng = item.findAll('tmX')
        
        try:
            sql = "INSERT INTO public_seoulbusarsid_info (arsId, stId, stNm, lat, lng) VALUES (?,?,?,?,?)"
            val = (arsId[0].text, stId[0].text, stNm[0].text, lat[0].text, lng[0].text)
        except IndexError:
            print(item)
            sql = "INSERT INTO public_seoulbusarsid_info (arsId, stId, stNm, lat, lng) VALUES (?,?,?,?,?)"
            val = (arsId[0].text, stId[0].text, stNm[0].text, lat[0].text, lng[0].text)
            
        cursor.execute(sql, val)
        mydb.commit()

조회날짜 : 20210917 | 순번 : 1 | start 삼각산동삼각산아이원아파트후문 public_seoulbusstts
조회날짜 : 20210917 | 순번 : 2 | start 삼각산동삼각산아이원아파트후문 public_seoulbusstts
조회날짜 : 20210917 | 순번 : 3 | start 롯데백화점미아점 public_seoulbusstts
('롯데백화점미아점',) 번째부터 조회수 제한이 걸렸습니다.
